In [84]:
import numpy as np
import pandas as pd
import statsmodels.discrete.conditional_models
import sklearn.linear_model
import pylogit

In [113]:
ketchup = pd.read_csv("datasets/dataset02.csv")
# print(ketchup.head())
ketchup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       2000 non-null   int64  
 1   Heinz40          2000 non-null   int64  
 2   Heinz32          2000 non-null   int64  
 3   Heinz28          2000 non-null   int64  
 4   Hunts32          2000 non-null   int64  
 5   dis_Heinz40      2000 non-null   int64  
 6   dis_Heinz32      2000 non-null   int64  
 7   dis_Heinz28      2000 non-null   int64  
 8   dis_Hunts32      2000 non-null   int64  
 9   feature_Heinz40  2000 non-null   int64  
 10  feature_Heinz32  2000 non-null   int64  
 11  feature_Heinz28  2000 non-null   int64  
 12  feature_Hunts32  2000 non-null   int64  
 13  price_Heinz40    2000 non-null   float64
 14  price_Heinz32    2000 non-null   float64
 15  price_Heinz28    2000 non-null   float64
 16  price_Hunts32    2000 non-null   float64
 17  brandsize_f   

In [117]:
ketchup[['av_Heinz40', 'av_Heinz32', 'av_Heinz28', 'av_Hunts32']] = 1
ketchup

,Unnamed: 0,Heinz40,Heinz32,Heinz28,Hunts32,dis_Heinz40,dis_Heinz32,dis_Heinz28,dis_Hunts32,feature_Heinz40,...,price_Heinz32,price_Heinz28,price_Hunts32,brandsize_f,obs_id,av_Heinz40,av_Heinz32,av_Heinz28,av_Hunts32,choice
0,1861,0,1,0,0,0,0,0,0,0,...,0.034,0.053,0.029,Heinz32,1861,1,1,1,1,1
1,282,0,1,0,0,0,0,0,0,0,...,0.030,0.046,0.041,Heinz32,282,1,1,1,1,1
2,110,0,1,0,0,0,0,0,0,0,...,0.033,0.046,0.048,Heinz32,110,1,1,1,1,1
3,914,1,0,0,0,0,0,0,0,0,...,0.031,0.047,0.031,Heinz40,914,1,1,1,1,2
4,1265,0,1,0,0,0,0,0,0,0,...,0.022,0.047,0.034,Heinz32,1265,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,94,0,1,0,0,0,0,0,0,0,...,0.022,0.042,0.034,Heinz32,94,1,1,1,1,1
1996,555,0,0,1,0,0,0,0,0,0,...,0.036,0.046,0.037,Heinz28,555,1,1,1,1,0
1997,807,0,0,1,0,0,0,0,0,0,...,0.030,0.039,0.030,Heinz28,807,1,1,1,1,0
1998,1589,0,1,0,0,0,1,0,0,0,...,0.031,0.043,0.034,Heinz32,1589,1,1,1,1,1


In [115]:
from statsmodels.discrete.conditional_models import ConditionalMNLogit
from pylogit.conditional_logit import MNL

y = pd.Series(np.where(ketchup[['Heinz28', 'Heinz32', 'Heinz40', 'Hunts32']]!=0)[1])

X = ketchup.loc[:, "price_Heinz40":"price_Hunts32"]

# categories = pd.Series(ketchup.columns[np.where(ketchup.loc[:, "Heinz40":"Hunts32"]!=0)[1]+1])
# categories = ketchup.index

model = ConditionalMNLogit(y, X, groups=ketchup["brandsize_f"])
model
# print(model.data)
# model.fit()
# categories
ketchup['choice'] = y
ketchup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       2000 non-null   int64  
 1   Heinz40          2000 non-null   int64  
 2   Heinz32          2000 non-null   int64  
 3   Heinz28          2000 non-null   int64  
 4   Hunts32          2000 non-null   int64  
 5   dis_Heinz40      2000 non-null   int64  
 6   dis_Heinz32      2000 non-null   int64  
 7   dis_Heinz28      2000 non-null   int64  
 8   dis_Hunts32      2000 non-null   int64  
 9   feature_Heinz40  2000 non-null   int64  
 10  feature_Heinz32  2000 non-null   int64  
 11  feature_Heinz28  2000 non-null   int64  
 12  feature_Hunts32  2000 non-null   int64  
 13  price_Heinz40    2000 non-null   float64
 14  price_Heinz32    2000 non-null   float64
 15  price_Heinz28    2000 non-null   float64
 16  price_Hunts32    2000 non-null   float64
 17  brandsize_f   

c:\Users\GLau\CodeProjects\marketing-models\venv\lib\site-packages\statsmodels\discrete\conditional_models.py:80: UserWarning: Dropped 4 groups and 2000 observations for having no within-group variance
  warnings.warn(msg)


In [127]:
ind_variables = ketchup.columns.tolist()[0:1]

alt_varying_variables = {u'price': dict([(0, 'price_Heinz40'),
                                        (1, 'price_Heinz32'),
                                        (2, 'price_Heinz28'),
                                        (3, 'price_Hunts32')])}

availability_variables = {0: 'av_Heinz40',
                          1: 'av_Heinz32', 
                          2: 'av_Heinz28',
                          3: 'av_Hunts32'}

custom_alt_id = "brandsize_i"

obs_id_column = "obs_id"

choice_column = "choice"


# Perform the conversion to long-format
long_ketchup = pylogit.convert_wide_to_long(ketchup, 
                                           ind_variables, 
                                           alt_varying_variables, 
                                           availability_variables, 
                                           obs_id_column, 
                                           choice_column,
                                           new_alt_id_name=custom_alt_id)
# Look at the resulting long-format dataframe
long_ketchup

c:\Users\GLau\CodeProjects\marketing-models\venv\lib\site-packages\pylogit\choice_tools.py:454: UserWarning: Note, there are 24 variables in wide_data but the inputs ind_vars, alt_specific_vars, and subset_specific_vars only account for 11 variables.
  warnings.warn(msg.format(num_dataframe_vars, data_title) +


,obs_id,brandsize_i,choice,Unnamed: 0,price
0,1861,0,0,1861,0.046
1,1861,1,1,1861,0.034
2,1861,2,0,1861,0.053
3,1861,3,0,1861,0.029
4,282,0,0,282,0.048
...,...,...,...,...,...
7995,1589,3,0,1589,0.034
7996,2050,0,0,2050,0.049
7997,2050,1,1,2050,0.032
7998,2050,2,0,2050,0.050


In [129]:
from collections import OrderedDict

basic_specification = OrderedDict()
basic_names = OrderedDict()

basic_specification["intercept"] = [0, 1, 3]
basic_names["intercept"] = ['intercept Heinz40',
                            'intercept Heinz32',
                            'intercept Hunts32']

In [145]:
ketchup_mnl = pylogit.create_choice_model(data=long_ketchup,
                                        alt_id_col=custom_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=basic_specification,
                                        model_type="MNL",
                                        names=basic_names)

# Specify the initial values and method for the optimization.
ketchup_mnl.fit_mle(np.zeros(3))

# Look at the estimation results
ketchup_mnl.get_statsmodels_summary()

ValueError: All rows pertaining to a given choice situation must be contiguous. 
Rows pertaining to the following observation id's are not contiguous: 
[1861, 282, 1716, 1970, 2447, 2730, 1902, 1298, 850, 271, 2715, 1382, 2057, 1713, 2387, 2450, 2580, 922, 2568, 1560, 2731, 1969, 2762, 1551, 2032, 618, 2797, 1210, 2377, 1843, 2234, 629, 2240, 2463, 1882, 1995, 1975, 1831, 679, 2700, 2335, 2347, 2325, 1614, 2626, 2757, 2689, 827, 339, 2305, 1044, 2389, 1573, 1718, 2093, 2172, 1769, 1289, 2724, 1628, 2561, 1837, 1916, 447, 2333, 2241, 791, 2398, 1893, 2739, 1961, 1409, 1412, 1176, 2624, 2551, 2538, 1510, 2242, 1896, 2348, 1466, 1597, 2225, 2595, 2706, 2345, 895, 2499, 2615, 1624, 2779, 583, 1681, 2328, 1493, 723, 2386, 2760, 1558, 2248, 2777, 2095, 1839, 2316, 1568, 2533, 1255, 2151, 2126, 1418, 2262, 1351, 1736, 2003, 2729, 322, 1169, 2503, 2209, 1013, 2559, 1500, 2758, 1852, 2185, 1948, 2782, 2733, 2148, 2359, 2603, 683, 460, 1967, 2180, 2529, 2145, 1996, 1675, 1973, 2681, 2506, 2514, 